<a href="https://colab.research.google.com/github/werowe/HypatiaAcademy/blob/master/pandas/pandas_missing_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to Clean Up Pandas Data

Here we should how to clean up Pandas data, in particular what do with about missing data, as well as what to do with invalid data.

Consider a survey.  If you send people a survey you cannot control in all cases what questions they will answer.  And you cannot anticipate what data they might put that is invalid.  For example, they might leave off some data.  Or they might enter a number in a question that is only supposed to be text.

So here we show you how to :

* generate some random data that is purposefully not clean
* drop duplicate rows
* get rid of rows that have missing values
* convert missing values to something else, like a fixed value of the average of all the other values in the colummn
* delete outliers, which are obvious typos.  For example, here we enter some salaries as $1 million while everyone else is around $100,000.  So that's most likely a mistake (since these are employees and not company owners or the CEO).
* apply a custom function to every row to do whatever special checking you want
* check for missing values
* show different ways to check for numbers or strings and how to convert those when they are of the wrong type

In [41]:
import numpy as np
import pandas as pd

import random

mean=10000
std=25



cols = [("name", str), ("income",np.int8), ("education", str),
     ("age", np.int8), ("city",str), ("id", np.int8), ("email", str), ("salary", np.int8),
        ("citizen", ["Y", "N"])]

words = [np.NaN, "", "abc", "def", "ghi", "jkl", "mno", "pqr"]

records = []


for i in range(20):

  data = {}

  for c in cols:

    if c[1] == np.int8:
      if random.randint(0,5)==5:
          data[c[0]] = np.NaN
      else:
          data[c[0]] = abs(int(random.gauss(mean, std)))

    if c[0] == "citizen":
       if random.randint(0,5)==5:
          data[c[0]] = random.randint(0,10)
       else:
          data[c[0]] =c[1][random.randint(0,1)]

    if c[1] == str and c[0] != "citizen":
       data[c[0]] = words[random.randint(0,len(words)-1)]

    if (c[0] == "income") & (random.randint(0,5)==0):
        data[c[0]] = chr(random.randint(65,70))

    if (c[0] == "income") & (random.randint(0,10)==0):
        data[c[0]] = 1000000

    if (c[0] == "age"):
        data[c[0]] = random.randint(20,65)

  records.append(data)

df=pd.DataFrame(records)


df

,name,income,education,age,city,id,email,salary,citizen
0,jkl,9980,abc,62,def,9977.0,,9995.0,Y
1,abc,9995,jkl,54,abc,9972.0,jkl,10012.0,Y
2,pqr,9987,ghi,40,jkl,9982.0,,9981.0,Y
3,def,10006,NaN,65,jkl,10031.0,abc,9992.0,Y
4,def,10024,mno,58,NaN,10009.0,pqr,NaN,N
5,pqr,NaN,,34,NaN,10038.0,mno,9989.0,Y
6,,NaN,NaN,22,jkl,10018.0,abc,10037.0,1
7,ghi,10020,abc,43,NaN,10018.0,abc,10018.0,Y
8,NaN,NaN,abc,40,def,9993.0,NaN,9996.0,Y
9,NaN,9998,,46,abc,10014.0,mno,10026.0,Y


In [42]:
# check if column in series is empty

df['name'].isnull()


0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8      True
9      True
10    False
11    False
12     True
13    False
14    False
15     True
16     True
17     True
18    False
19    False
Name: name, dtype: bool

In [43]:
# draw from mean()

sMean=df['salary'].mean()

df['salary'] = df['salary'].fillna(sMean)

df

,name,income,education,age,city,id,email,salary,citizen
0,jkl,9980,abc,62,def,9977.0,,9995.000000,Y
1,abc,9995,jkl,54,abc,9972.0,jkl,10012.000000,Y
2,pqr,9987,ghi,40,jkl,9982.0,,9981.000000,Y
3,def,10006,NaN,65,jkl,10031.0,abc,9992.000000,Y
4,def,10024,mno,58,NaN,10009.0,pqr,10003.263158,N
5,pqr,NaN,,34,NaN,10038.0,mno,9989.000000,Y
6,,NaN,NaN,22,jkl,10018.0,abc,10037.000000,1
7,ghi,10020,abc,43,NaN,10018.0,abc,10018.000000,Y
8,NaN,NaN,abc,40,def,9993.0,NaN,9996.000000,Y
9,NaN,9998,,46,abc,10014.0,mno,10026.000000,Y


In [44]:
# Yes, No columns cannot contain numbers
# NaN is OK as we can get rid of it in a second step plus certain functions like mean()
# will ignore it.  coerce means convert to NaN on error

df['income'] = df['income'].apply(lambda x: pd.to_numeric(x, errors='coerce'))

df['income']

0        9980.0
1        9995.0
2        9987.0
3       10006.0
4       10024.0
5           NaN
6           NaN
7       10020.0
8           NaN
9        9998.0
10       9977.0
11          NaN
12          NaN
13      10006.0
14          NaN
15      10021.0
16          NaN
17       9981.0
18    1000000.0
19       9994.0
Name: income, dtype: float64

In [45]:
# drop outliers, too many standard deviations away


mean = df['income'].mean()
std_dev = df['income'].std()

# Define threshold for outliers (e.g., values more than 2 standard deviations away from the mean)
threshold = 2

df.loc[(df['income'] - mean) > 2 * std_dev]



,name,income,education,age,city,id,email,salary,citizen
18,pqr,1000000.0,def,45,def,10004.0,def,10006.0,N


In [46]:
df.drop(df.loc[(df['income'] - mean) > 2 * std_dev].index, inplace=True)

df

,name,income,education,age,city,id,email,salary,citizen
0,jkl,9980.0,abc,62,def,9977.0,,9995.000000,Y
1,abc,9995.0,jkl,54,abc,9972.0,jkl,10012.000000,Y
2,pqr,9987.0,ghi,40,jkl,9982.0,,9981.000000,Y
3,def,10006.0,NaN,65,jkl,10031.0,abc,9992.000000,Y
4,def,10024.0,mno,58,NaN,10009.0,pqr,10003.263158,N
5,pqr,NaN,,34,NaN,10038.0,mno,9989.000000,Y
6,,NaN,NaN,22,jkl,10018.0,abc,10037.000000,1
7,ghi,10020.0,abc,43,NaN,10018.0,abc,10018.000000,Y
8,NaN,NaN,abc,40,def,9993.0,NaN,9996.000000,Y
9,NaN,9998.0,,46,abc,10014.0,mno,10026.000000,Y


In [47]:
# drop all numbers in Yes/No answer

df['citizen'] = df['citizen'].apply(lambda x: x if isinstance(x,str) else np.NaN)
df['citizen']


0       Y
1       Y
2       Y
3       Y
4       N
5       Y
6     NaN
7       Y
8       Y
9       Y
10      N
11      Y
12      Y
13    NaN
14    NaN
15    NaN
16      Y
17      N
19    NaN
Name: citizen, dtype: object

In [48]:
df

,name,income,education,age,city,id,email,salary,citizen
0,jkl,9980.0,abc,62,def,9977.0,,9995.000000,Y
1,abc,9995.0,jkl,54,abc,9972.0,jkl,10012.000000,Y
2,pqr,9987.0,ghi,40,jkl,9982.0,,9981.000000,Y
3,def,10006.0,NaN,65,jkl,10031.0,abc,9992.000000,Y
4,def,10024.0,mno,58,NaN,10009.0,pqr,10003.263158,N
5,pqr,NaN,,34,NaN,10038.0,mno,9989.000000,Y
6,,NaN,NaN,22,jkl,10018.0,abc,10037.000000,NaN
7,ghi,10020.0,abc,43,NaN,10018.0,abc,10018.000000,Y
8,NaN,NaN,abc,40,def,9993.0,NaN,9996.000000,Y
9,NaN,9998.0,,46,abc,10014.0,mno,10026.000000,Y


In [49]:
# now we are left with only those rows that have all non-Nan columns.  Note that we still
# have some blanks sells.  You would choose the various procedures we have explained above
# to determine how you want to handle this situation.

df.dropna(how='any', inplace=True)
df

,name,income,education,age,city,id,email,salary,citizen
0,jkl,9980.0,abc,62,def,9977.0,,9995.0,Y
1,abc,9995.0,jkl,54,abc,9972.0,jkl,10012.0,Y
2,pqr,9987.0,ghi,40,jkl,9982.0,,9981.0,Y
10,def,9977.0,jkl,23,pqr,9966.0,,9985.0,N


In [50]:
# here we operate on a Pandas series, which is a single column.  But since we are
# sending each row one row at a time we are operating on a single row-column combination
# This s is a single value, a scalar.  So we can work with it as we would with any
# Python primitive (meaning built-in type.  Pandas and Numpy are extension of Python into new types.)


def toUpper(s):
    if isinstance(s,str):
      return s.upper()


df['city']=df['city'].apply(toUpper)
df

,name,income,education,age,city,id,email,salary,citizen
0,jkl,9980.0,abc,62,DEF,9977.0,,9995.0,Y
1,abc,9995.0,jkl,54,ABC,9972.0,jkl,10012.0,Y
2,pqr,9987.0,ghi,40,JKL,9982.0,,9981.0,Y
10,def,9977.0,jkl,23,PQR,9966.0,,9985.0,N


In [51]:
# axis = 1 means row
# axis = 0 means column

# the important point to note there is we send the entire row in as a paramters. thus all the columns
# are available to use.  remember to send back the entire row after you have updated any of the columns.

def wholeRow(row):
  if isinstance(row['city'],str):
     row['city'] = row['city'].lower()
     return row


df=df.apply(wholeRow, axis=1)
df

,name,income,education,age,city,id,email,salary,citizen
0,jkl,9980.0,abc,62,def,9977.0,,9995.0,Y
1,abc,9995.0,jkl,54,abc,9972.0,jkl,10012.0,Y
2,pqr,9987.0,ghi,40,jkl,9982.0,,9981.0,Y
10,def,9977.0,jkl,23,pqr,9966.0,,9985.0,N


In [52]:
df

,name,income,education,age,city,id,email,salary,citizen
0,jkl,9980.0,abc,62,def,9977.0,,9995.0,Y
1,abc,9995.0,jkl,54,abc,9972.0,jkl,10012.0,Y
2,pqr,9987.0,ghi,40,jkl,9982.0,,9981.0,Y
10,def,9977.0,jkl,23,pqr,9966.0,,9985.0,N


In [53]:
'''
 eleminate duplicates.  the way to do that is to group all columns and sum.  If the count is bigger then one them delete all but one
 '''


'\n eleminate duplicates.  the way to do that is to group all columns and sum.  If the count is bigger then one them delete all but one\n '